In [1]:
pip install pandas numpy python-chess tqdm requests zstandard

Note: you may need to restart the kernel to use updated packages.


# data preprocessing

In [2]:
import pandas as pd
import numpy as np
import chess
import chess.pgn
from pathlib import Path
import requests
import zstandard as zstd
from tqdm import tqdm
import time
import io


In [3]:
def download_chess_database():
    """
    Загрузка базы данных шахматных партий
    """
    url = "https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst"
    filename = "lichess_db_standard_rated_2013-01.pgn.zst"
    
    if not Path(filename).exists():
        print("Загрузка базы данных...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        
        with open(filename, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='iB',
            unit_scale=True
        ) as pbar:
            for data in response.iter_content(chunk_size=1024):
                size = f.write(data)
                pbar.update(size)
    else:
        print("База данных уже загружена")

In [4]:
def create_output_directory(base_path="preprocessed_data"):
    """
    Создание директории для сохранения результатов
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для результатов: {output_dir.absolute()}")
    return output_dir

In [5]:
def calculate_piece_square_value(board, color):
    """
    Расчет позиционной ценности фигур на основе piece-square tables
    """
    pawn_table = [0, 0, 0, 0, 0, 0, 0, 0,
                  5, 5, 5, 5, 5, 5, 5, 5,
                  1, 1, 2, 3, 3, 2, 1, 1,
                  0, 0, 1, 2, 2, 1, 0, 0,
                  0, 0, 0, 2, 2, 0, 0, 0,
                  0, 0, 0, 1, 1, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0]
    
    knight_table = [0, 1, 2, 2, 2, 2, 1, 0,
                    1, 2, 3, 3, 3, 3, 2, 1,
                    2, 3, 4, 4, 4, 4, 3, 2,
                    2, 3, 4, 5, 5, 4, 3, 2,
                    2, 3, 4, 5, 5, 4, 3, 2,
                    2, 3, 4, 4, 4, 4, 3, 2,
                    1, 2, 3, 3, 3, 3, 2, 1,
                    0, 1, 2, 2, 2, 2, 1, 0]
    
    value = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            if piece.piece_type == chess.PAWN:
                value += pawn_table[square if color == chess.WHITE else 63 - square]
            elif piece.piece_type == chess.KNIGHT:
                value += knight_table[square if color == chess.WHITE else 63 - square]
    
    return value

In [6]:
def calculate_king_safety(board, color):
    """
    Расчет безопасности короля
    """
    king_square = board.king(color)
    if king_square is None:
        return -10  
    
    safety = 0
    
    if color == chess.WHITE:
        pawn_shield_squares = [king_square + 8, king_square + 7, king_square + 9]
    else:
        pawn_shield_squares = [king_square - 8, king_square - 7, king_square - 9]
    
    for square in pawn_shield_squares:
        if 0 <= square <= 63:
            piece = board.piece_at(square)
            if piece and piece.piece_type == chess.PAWN and piece.color == color:
                safety += 1
    
    king_file = chess.square_file(king_square)
    for file_offset in [-1, 0, 1]:
        file = king_file + file_offset
        if 0 <= file <= 7:
            has_pawn = False
            for rank in range(8):
                square = chess.square(file, rank)
                piece = board.piece_at(square)
                if piece and piece.piece_type == chess.PAWN and piece.color == color:
                    has_pawn = True
                    break
            if not has_pawn:
                safety -= 2
    
    return safety

In [7]:
def calculate_pawn_structure(board, color):
    """
    Оценка пешечной структуры
    """
    score = 0
    pawns = board.pieces(chess.PAWN, color)

    for pawn_square in pawns:
        file = chess.square_file(pawn_square)
        isolated = True
        for adjacent_file in [file - 1, file + 1]:
            if 0 <= adjacent_file <= 7:
                for rank in range(8):
                    square = chess.square(adjacent_file, rank)
                    piece = board.piece_at(square)
                    if piece and piece.piece_type == chess.PAWN and piece.color == color:
                        isolated = False
                        break
                if not isolated:
                    break
        if isolated:
            score -= 1
    
    file_counts = [0] * 8
    for pawn_square in pawns:
        file = chess.square_file(pawn_square)
        file_counts[file] += 1
    
    for count in file_counts:
        if count > 1:
            score -= (count - 1)
    
    for pawn_square in pawns:
        file = chess.square_file(pawn_square)
        rank = chess.square_rank(pawn_square)
        
        is_passed = True
        if color == chess.WHITE:
            for check_rank in range(rank + 1, 8):
                for check_file in [file - 1, file, file + 1]:
                    if 0 <= check_file <= 7:
                        square = chess.square(check_file, check_rank)
                        piece = board.piece_at(square)
                        if piece and piece.piece_type == chess.PAWN and piece.color != color:
                            is_passed = False
                            break
                if not is_passed:
                    break
        else:
            for check_rank in range(rank - 1, -1, -1):
                for check_file in [file - 1, file, file + 1]:
                    if 0 <= check_file <= 7:
                        square = chess.square(check_file, check_rank)
                        piece = board.piece_at(square)
                        if piece and piece.piece_type == chess.PAWN and piece.color != color:
                            is_passed = False
                            break
                if not is_passed:
                    break
        
        if is_passed:
            score += 2
    
    return score

In [8]:
def extract_move_features(game, move_number):
    """
    Извлечение улучшенных признаков для конкретного хода
    """
    board = chess.Board()
    features = []
    
    for i, move in enumerate(game.mainline_moves()):
        if i >= move_number:
            break
        board.push(move)
    
    pawn_diff = len(board.pieces(chess.PAWN, chess.WHITE)) - len(board.pieces(chess.PAWN, chess.BLACK))
    knight_diff = len(board.pieces(chess.KNIGHT, chess.WHITE)) - len(board.pieces(chess.KNIGHT, chess.BLACK))
    bishop_diff = len(board.pieces(chess.BISHOP, chess.WHITE)) - len(board.pieces(chess.BISHOP, chess.BLACK))
    rook_diff = len(board.pieces(chess.ROOK, chess.WHITE)) - len(board.pieces(chess.ROOK, chess.BLACK))
    queen_diff = len(board.pieces(chess.QUEEN, chess.WHITE)) - len(board.pieces(chess.QUEEN, chess.BLACK))
    
    material_value = pawn_diff * 1 + knight_diff * 3 + bishop_diff * 3 + rook_diff * 5 + queen_diff * 9
    
    center_squares = [chess.E4, chess.E5, chess.D4, chess.D5]
    extended_center = [chess.C3, chess.C4, chess.C5, chess.C6, chess.D3, chess.D6, 
                      chess.E3, chess.E6, chess.F3, chess.F4, chess.F5, chess.F6]
    
    center_control = 0
    extended_center_control = 0
    
    for square in center_squares:
        piece = board.piece_at(square)
        if piece:
            if piece.color == chess.WHITE:
                center_control += 2
            else:
                center_control -= 2
        if board.is_attacked_by(chess.WHITE, square):
            center_control += 1
        if board.is_attacked_by(chess.BLACK, square):
            center_control -= 1
    
    for square in extended_center:
        piece = board.piece_at(square)
        if piece:
            if piece.color == chess.WHITE:
                extended_center_control += 1
            else:
                extended_center_control -= 1
    
    white_mobility = len(list(board.legal_moves)) if board.turn == chess.WHITE else 0
    board.turn = not board.turn
    black_mobility = len(list(board.legal_moves)) if board.turn == chess.BLACK else 0
    board.turn = not board.turn
    mobility_diff = white_mobility - black_mobility
    
    white_king_safety = calculate_king_safety(board, chess.WHITE)
    black_king_safety = calculate_king_safety(board, chess.BLACK)
    king_safety_diff = white_king_safety - black_king_safety
    
    white_development = 0
    black_development = 0
    
    for piece_type in [chess.KNIGHT, chess.BISHOP]:
        for square in board.pieces(piece_type, chess.WHITE):
            if chess.square_rank(square) > 1:  
                white_development += 1
                if square in center_squares or square in extended_center:
                    white_development += 0.5  
        
        for square in board.pieces(piece_type, chess.BLACK):
            if chess.square_rank(square) < 6:  
                black_development += 1
                if square in center_squares or square in extended_center:
                    black_development += 0.5
    
    development_diff = white_development - black_development
    
    white_pawn_structure = calculate_pawn_structure(board, chess.WHITE)
    black_pawn_structure = calculate_pawn_structure(board, chess.BLACK)
    pawn_structure_diff = white_pawn_structure - black_pawn_structure
    
    white_piece_square = calculate_piece_square_value(board, chess.WHITE)
    black_piece_square = calculate_piece_square_value(board, chess.BLACK)
    piece_square_diff = white_piece_square - black_piece_square
    
    important_squares = [chess.E4, chess.E5, chess.D4, chess.D5, chess.F7, chess.F2]
    square_control = 0
    for square in important_squares:
        if board.is_attacked_by(chess.WHITE, square):
            square_control += 1
        if board.is_attacked_by(chess.BLACK, square):
            square_control -= 1
    
    total_pieces = len(board.piece_map())
    game_phase = min(total_pieces / 32.0, 1.0)  
    
    tempo = move_number / 40.0  
    
    features.extend([
        pawn_diff, knight_diff, bishop_diff, rook_diff, queen_diff,  
        material_value,  
        center_control, extended_center_control,  
        mobility_diff,  
        king_safety_diff,  
        development_diff,  
        pawn_structure_diff,  
        piece_square_diff,  
        square_control,  
        game_phase,  
        tempo,  
        white_mobility, black_mobility,  
        white_king_safety, black_king_safety,  
        white_development, black_development,  
        white_pawn_structure, black_pawn_structure,  
        white_piece_square, black_piece_square  
    ])
    
    return features

In [9]:
def preprocess_chess_database(file_path, max_games=100):
    """
    Предобработка базы данных шахматных партий с фильтрацией качественных игр
    """
    print("Начало предобработки данных...")
    start_time = time.time()
    
    features_list = []
    labels_list = []
    game_metadata = []
    
    result_counts = {1: 0, -1: 0, 0: 0}  
    max_white_wins = int(max_games * 0.33) 
    max_black_wins = int(max_games * 0.33)  
    max_draws = int(max_games * 0.33)      
    
    max_per_result = {1: max_white_wins, -1: max_black_wins, 0: max_draws}
    
    with open(file_path, 'rb') as fh:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(fh) as reader:
            text_reader = io.TextIOWrapper(reader, encoding='utf-8')
            pgn = chess.pgn.read_game(text_reader)
            game_count = 0
            processed_games = 0
            
            while pgn and (result_counts[1] < max_white_wins or result_counts[-1] < max_black_wins or result_counts[0] < max_draws):
                result = pgn.headers.get('Result', '*')
                if result == '1-0':
                    result_value = 1  
                elif result == '0-1':
                    result_value = -1  
                elif result == '1/2-1/2':
                    result_value = 0  
                else:
                    pgn = chess.pgn.read_game(text_reader)
                    continue  
                
                white_elo = pgn.headers.get('WhiteElo', '0')
                black_elo = pgn.headers.get('BlackElo', '0')
                time_control = pgn.headers.get('TimeControl', '')
                
                try:
                    white_elo = int(white_elo) if white_elo.isdigit() else 0
                    black_elo = int(black_elo) if black_elo.isdigit() else 0
                except:
                    white_elo = black_elo = 0

                if white_elo < 1200 or black_elo < 1200:
                    pgn = chess.pgn.read_game(text_reader)
                    continue
                
                if abs(white_elo - black_elo) > 400:
                    pgn = chess.pgn.read_game(text_reader)
                    continue
                
                if 'bullet' in time_control.lower() or ('60+' in time_control and '0' in time_control):
                    pgn = chess.pgn.read_game(text_reader)
                    continue
                
                if result_counts[result_value] >= max_per_result[result_value]:
                    pgn = chess.pgn.read_game(text_reader)
                    continue
                
                move_list = list(pgn.mainline_moves())
                if len(move_list) < 10 or len(move_list) > 100:  
                    pgn = chess.pgn.read_game(text_reader)
                    continue
                
                total_moves = len(move_list)
                
                move_indices = []
                
                for i in range(5, min(15, total_moves), 2):
                    move_indices.append(i)
                
                if total_moves > 15:
                    for i in range(15, min(40, total_moves), 3):
                        move_indices.append(i)
                
                if total_moves > 40:
                    for i in range(max(40, total_moves - 15), total_moves, 2):
                        move_indices.append(i)
                
                move_indices = move_indices[:15]
                
                for move_idx in move_indices:
                    if move_idx < total_moves:
                        features = extract_move_features(pgn, move_idx)
                        features_list.append(features)
                        labels_list.append(result_value)
                        game_metadata.append({
                            'game_id': processed_games,
                            'move_number': move_idx,
                            'white_elo': white_elo,
                            'black_elo': black_elo,
                            'game_phase': 'opening' if move_idx < 15 else ('middlegame' if move_idx < 40 else 'endgame')
                        })
                
                result_counts[result_value] += 1
                processed_games += 1
                
                if processed_games % 100 == 0:
                    print(f"Обработано {processed_games} игр. Распределение: Белые: {result_counts[1]}, Черные: {result_counts[-1]}, Ничьи: {result_counts[0]}")
                
                pgn = chess.pgn.read_game(text_reader)
    
    df = pd.DataFrame(features_list, columns=[
        'pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',  
        'material_value',  
        'center_control', 'extended_center_control',  
        'mobility_diff',  
        'king_safety_diff',  
        'development_diff',  
        'pawn_structure_diff',  
        'piece_square_diff',  
        'square_control',  
        'game_phase',  
        'tempo',  
        'white_mobility', 'black_mobility',  
        'white_king_safety', 'black_king_safety',  
        'white_development', 'black_development',  
        'white_pawn_structure', 'black_pawn_structure',  
        'white_piece_square', 'black_piece_square'  
    ])
    df['result'] = labels_list
    
    print(f"\nПредобработка завершена за {time.time() - start_time:.2f} секунд")
    print(f"Обработано {processed_games} игр")
    print(f"Получено {len(df)} позиций")
    print(f"Финальное распределение результатов:")
    print(f"  Победы белых: {result_counts[1]} игр")
    print(f"  Победы черных: {result_counts[-1]} игр") 
    print(f"  Ничьи: {result_counts[0]} игр")
    
    metadata_df = pd.DataFrame(game_metadata)
    df = pd.concat([df, metadata_df], axis=1)
    
    return df

In [10]:
def main():
    output_dir = create_output_directory()
    
    db_file = "lichess_db_standard_rated_2013-01.pgn.zst"
    if not Path(db_file).exists():
        download_chess_database()
    
    df = preprocess_chess_database(db_file)

    output_file = output_dir / "processed_chess_data.csv"
    df.to_csv(output_file, index=False)
    print(f"\nДанные сохранены в {output_file}")
    
    report_file = output_dir / "data_report.txt"
    with open(report_file, 'w') as f:
        f.write("Отчет о предобработке данных\n")
        f.write("==========================\n\n")
        f.write(f"Количество позиций: {len(df)}\n")
        f.write(f"Количество признаков: {len(df.columns) - 1}\n")
        f.write("\nСтатистика по результатам:\n")
        f.write(df['result'].value_counts().to_string())
        f.write("\n\nСтатистика по признакам:\n")
        f.write(df.describe().to_string())


In [11]:
if __name__ == "__main__":
    main() 

Создана директория для результатов: /Users/skolkoff/Desktop/StoneS_ЭП/preprocessed_data
Начало предобработки данных...

Предобработка завершена за 3.34 секунд
Обработано 99 игр
Получено 1358 позиций
Финальное распределение результатов:
  Победы белых: 33 игр
  Победы черных: 33 игр
  Ничьи: 33 игр

Данные сохранены в preprocessed_data/processed_chess_data.csv


# exploratory analysis

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [13]:
def create_output_directory(base_path="exploratory_analysis_results"):
    """
    Создание директории для сохранения результатов анализа
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для результатов: {output_dir.absolute()}")
    return output_dir

In [14]:
def load_data():
    """
    Загрузка предобработанных данных
    """
    df = pd.read_csv('preprocessed_data/processed_chess_data.csv')
    return df


In [15]:
def analyze_feature_distributions(df, output_dir):
    """
    Анализ распределений признаков
    """
    print("\nАнализ распределений признаков...")
    
    features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                'square_control', 'game_phase', 'tempo']
    
    available_features = [f for f in features if f in df.columns]
    if len(available_features) != len(features):
        print(f"Внимание: Доступно только {len(available_features)} из {len(features)} признаков")
        features = available_features
    
    numeric_features = []
    categorical_features = []
    
    for feature in features:
        if df[feature].dtype in ['object', 'string']:
            categorical_features.append(feature)
        else:
            numeric_features.append(feature)
    
    for feature in numeric_features:
        plt.figure(figsize=(10, 6))
        sns.histplot(data=df, x=feature, bins=30)
        plt.title(f'Распределение признака {feature}')
        plt.savefig(output_dir / f'{feature}_distribution.png')
        plt.close()
    
    for feature in categorical_features:
        plt.figure(figsize=(10, 6))
        df[feature].value_counts().plot(kind='bar')
        plt.title(f'Распределение признака {feature}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(output_dir / f'{feature}_distribution.png')
        plt.close()
    
    if numeric_features:
        plt.figure(figsize=(12, 8))
        correlation_matrix = df[numeric_features].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
        plt.title('Корреляционная матрица числовых признаков')
        plt.tight_layout()
        plt.savefig(output_dir / 'correlation_matrix.png')
        plt.close()

In [16]:
def analyze_move_patterns(df, output_dir):
    """
    Анализ паттернов ходов
    """
    print("\nАнализ паттернов ходов...")
    
    if 'material_value' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='material_value')
        plt.title('Распределение материального преимущества по результатам')
        plt.savefig(output_dir / 'material_advantage_by_result.png')
        plt.close()
    
    if 'center_control' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='center_control')
        plt.title('Распределение контроля центра по результатам')
        plt.savefig(output_dir / 'center_control_by_result.png')
        plt.close()
    
    if 'mobility_diff' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='mobility_diff')
        plt.title('Распределение разности мобильности по результатам')
        plt.savefig(output_dir / 'mobility_diff_by_result.png')
        plt.close()
    
    if 'king_safety_diff' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='king_safety_diff')
        plt.title('Распределение безопасности короля по результатам')
        plt.savefig(output_dir / 'king_safety_by_result.png')
        plt.close()

In [17]:
def perform_pca_analysis(df, output_dir):
    """
    Анализ главных компонент
    """
    print("\nВыполнение PCA анализа...")
    
    features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                'square_control', 'game_phase', 'tempo']
    
    available_features = [f for f in features if f in df.columns]
    if len(available_features) != len(features):
        print(f"Внимание: Доступно только {len(available_features)} из {len(features)} признаков")
        features = available_features
    
    numeric_features = []
    for feature in features:
        if df[feature].dtype in ['int64', 'float64']:
            numeric_features.append(feature)
    
    if not numeric_features:
        print("Нет числовых признаков для PCA анализа")
        return None
    
    X = df[numeric_features]
    features = numeric_features  
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Количество компонент')
    plt.ylabel('Объясненная дисперсия')
    plt.title('Кумулятивная объясненная дисперсия')
    plt.savefig(output_dir / 'pca_explained_variance.png')
    plt.close()
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['result'], cmap='viridis')
    plt.colorbar(scatter, label='Результат')
    plt.xlabel('Первая главная компонента')
    plt.ylabel('Вторая главная компонента')
    plt.title('PCA: первые две компоненты')
    plt.savefig(output_dir / 'pca_first_two_components.png')
    plt.close()
    
    component_info = pd.DataFrame(
        pca.components_,
        columns=features,
        index=[f'PC{i+1}' for i in range(len(features))]
    )
    component_info.to_csv(output_dir / 'pca_components.csv')

In [18]:
def generate_analysis_report(df, output_dir):
    """
    Генерация отчета по анализу
    """
    print("\nГенерация отчета...")
    
    with open(output_dir / 'analysis_report.txt', 'w') as f:
        f.write("Отчет по исследовательскому анализу данных\n")
        f.write("=======================================\n\n")
        
        f.write("1. Общая информация о данных:\n")
        f.write(f"Количество позиций: {len(df)}\n")
        f.write(f"Количество признаков: {len(df.columns) - 1}\n\n")
        
        f.write("2. Статистика по результатам:\n")
        f.write(df['result'].value_counts().to_string())
        f.write("\n\n")
        
        f.write("3. Описательная статистика признаков:\n")
        f.write(df.describe().to_string())
        f.write("\n\n")
        
        f.write("4. Корреляции между признаками:\n")
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
        exclude_columns = ['result', 'game_id', 'move_number', 'white_elo', 'black_elo']
        numeric_features = [col for col in numeric_columns if col not in exclude_columns]
        
        if numeric_features:
            f.write(df[numeric_features].corr().to_string())
        else:
            f.write("Числовые признаки не найдены")

In [19]:
def main():
    output_dir = create_output_directory()
    df = load_data()
    analyze_feature_distributions(df, output_dir)
    analyze_move_patterns(df, output_dir)
    perform_pca_analysis(df, output_dir)
    generate_analysis_report(df, output_dir)
    
    print(f"\nАнализ завершен. Результаты сохранены в {output_dir}")

In [20]:
if __name__ == "__main__":
    main() 

Создана директория для результатов: /Users/skolkoff/Desktop/StoneS_ЭП/exploratory_analysis_results

Анализ распределений признаков...

Анализ паттернов ходов...

Выполнение PCA анализа...

Генерация отчета...

Анализ завершен. Результаты сохранены в exploratory_analysis_results


# model training

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.class_weight import compute_class_weight
import time
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [22]:
def create_output_directory(base_path="model_figures"):
    """
    Создание директории для сохранения графиков
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для графиков: {output_dir.absolute()}")
    return output_dir

In [23]:
def load_and_prepare_data():
    """
    Загрузка и подготовка данных для обучения
    """
    try:
        df = pd.read_csv('preprocessed_data/processed_chess_data.csv')
        print(f"Загружено {len(df)} строк данных")
        
        features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                   'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                   'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                   'square_control', 'game_phase', 'tempo', 'white_mobility', 'black_mobility',
                   'white_king_safety', 'black_king_safety', 'white_development', 'black_development',
                   'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square']
        
        available_features = [f for f in features if f in df.columns]
        if len(available_features) != len(features):
            print(f"Внимание: Доступно только {len(available_features)} из {len(features)} признаков")
            print(f"Недостающие признаки: {set(features) - set(available_features)}")
        
        X = df[available_features]
        
        y = df['result'].map({-1: 0, 0: 1, 1: 2})
        
        print(f"Размерность признаков: {X.shape}")
        print(f"Размерность целевой переменной: {y.shape}")
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        print(f"Размер обучающей выборки: {X_train.shape}")
        print(f"Размер тестовой выборки: {X_test.shape}")
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test
    except Exception as e:
        print(f"Ошибка при загрузке данных: {str(e)}")
        raise

In [24]:
def perform_robust_cross_validation(model, X, y, cv_strategies):
    """
    Выполнение кросс-валидации с использованием различных стратегий
    """
    cv_results = {}
    
    for name, cv in cv_strategies.items():
        try:
            scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
            cv_results[name] = {
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
                'scores': scores
            }
        except Exception as e:
            print(f"Ошибка при выполнении {name} кросс-валидации: {str(e)}")
            cv_results[name] = {
                'mean_score': 0,
                'std_score': 0,
                'scores': []
            }
    
    return cv_results

In [25]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, output_dir):
    """
    Обучение и оценка различных моделей с балансировкой классов
    """
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(zip(np.unique(y_train), class_weights))
    
    print(f"Веса классов для балансировки: {class_weight_dict}")
    
    models = {
        'Логистическая регрессия': LogisticRegression(
            max_iter=2000, random_state=42, class_weight='balanced',
            C=0.1, solver='liblinear'
        ),
        'Дерево решений': DecisionTreeClassifier(
            random_state=42, class_weight='balanced',
            max_depth=10, min_samples_split=20, min_samples_leaf=10
        ),
        'Случайный лес': RandomForestClassifier(
            n_estimators=200, random_state=42, class_weight='balanced',
            max_depth=15, min_samples_split=10, min_samples_leaf=5,
            max_features='sqrt'
        ),
        'Градиентный бустинг': GradientBoostingClassifier(
            n_estimators=200, random_state=42, learning_rate=0.1,
            max_depth=6, min_samples_split=20, min_samples_leaf=10
        ),
        'AdaBoost': AdaBoostClassifier(
            n_estimators=100, algorithm='SAMME', random_state=42,
            learning_rate=0.8
        ),
        'SVM': SVC(
            random_state=42, class_weight='balanced',
            C=1.0, kernel='rbf', gamma='scale'
        ),
        'Нейронная сеть': MLPClassifier(
            hidden_layer_sizes=(200, 100, 50), max_iter=2000, random_state=42,
            alpha=0.01, learning_rate='adaptive', early_stopping=True,
            validation_fraction=0.1
        ),
        'KNN': KNeighborsClassifier(n_neighbors=7, weights='distance')
    }
    
    cv_strategies = {
        'KFold': KFold(n_splits=5, shuffle=True, random_state=42),
        'StratifiedKFold': StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    }
    
    results = []
    
    for name, model in models.items():
        try:
            print(f"\nОбучение модели: {name}")
            
            start_time = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start_time
            
            cv_results = perform_robust_cross_validation(model, X_train, y_train, cv_strategies)
            
            y_pred = model.predict(X_test)
            
            metrics = {
                'Модель': name,
                'Время обучения (с)': train_time,
                'Точность (accuracy)': accuracy_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
                'Recall': recall_score(y_test, y_pred, average='weighted'),
                'F1-score': f1_score(y_test, y_pred, average='weighted'),
                'KFold CV Score': cv_results['KFold']['mean_score'],
                'KFold CV Std': cv_results['KFold']['std_score'],
                'StratifiedKFold CV Score': cv_results['StratifiedKFold']['mean_score'],
                'StratifiedKFold CV Std': cv_results['StratifiedKFold']['std_score']
            }
            
            results.append(metrics)
            
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title(f'Матрица ошибок - {name}')
            plt.xlabel('Предсказанный класс')
            plt.ylabel('Истинный класс')
            plt.tight_layout()
            plt.savefig(output_dir / f'confusion_matrix_{name.lower().replace(" ", "_")}.png')
            plt.close()
            
            print(f"Время обучения: {train_time:.2f} секунд")
            print(f"Точность на тестовой выборке: {metrics['Точность (accuracy)']:.4f}")
            print(f"KFold CV Score: {metrics['KFold CV Score']:.4f} ± {metrics['KFold CV Std']:.4f}")
            print(f"StratifiedKFold CV Score: {metrics['StratifiedKFold CV Score']:.4f} ± {metrics['StratifiedKFold CV Std']:.4f}")
            print("\nClassification Report:")
            print(classification_report(y_test, y_pred, zero_division=0))
            
        except Exception as e:
            print(f"Ошибка при обучении модели {name}: {str(e)}")
            continue
    
    return pd.DataFrame(results)

In [26]:
def visualize_results(results_df, output_dir):
    """
    Визуализация результатов обучения моделей
    """
    try:
        metrics = ['Точность (accuracy)', 'Precision', 'Recall', 'F1-score']
        plt.figure(figsize=(12, 6))
        results_df.plot(x='Модель', y=metrics, kind='bar')
        plt.title('Сравнение метрик для разных моделей')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(output_dir /'model_metrics_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()

        plt.figure(figsize=(10, 6))
        sns.barplot(data=results_df, x='Модель', y='Время обучения (с)')
        plt.title('Время обучения моделей')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(output_dir /'training_time_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()

        plt.figure(figsize=(12, 6))
        results_df.plot(x='Модель', y=['KFold CV Score', 'StratifiedKFold CV Score'], kind='bar')
        plt.title('Сравнение результатов кросс-валидации')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(output_dir / 'cv_accuracy_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("\nСводная таблица результатов:")
        print(results_df.to_string(float_format='{:.4f}'.format))
        
    except Exception as e:
        print(f"Ошибка при визуализации результатов: {str(e)}")

In [27]:
class ChessMoveDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels.values if hasattr(labels, 'values') else labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [28]:
class ChessMoveNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.2):
        super(ChessMoveNetwork, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_size),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [29]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs, device, output_dir):
    print("\nНачало обучения модели...")
    
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    best_val_accuracy = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), output_dir / 'best_model.pth')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, '
              f'Val Accuracy: {val_accuracy:.2f}%')
    
    return train_losses, val_losses, train_accuracies, val_accuracies

In [30]:
def visualize_training_results(train_losses, val_losses, train_accuracies, 
                             val_accuracies, output_dir):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(output_dir / 'loss_plot.png')
    plt.close()
    plt.figure(figsize=(10, 6))
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.savefig(output_dir / 'accuracy_plot.png')
    plt.close()


In [31]:
def evaluate_model(model, test_loader, device, output_dir):
    print("\nОценка модели на тестовой выборке...")
    
    model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for features, labels in test_loader:
            features = features.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    confusion_matrix = pd.crosstab(
        pd.Series(all_labels, name='Actual'),
        pd.Series(all_predictions, name='Predicted')
    )
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.savefig(output_dir / 'confusion_matrix.png')
    plt.close()
    
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    print(f"\nTest Accuracy: {accuracy:.4f}")
    
    return accuracy

In [32]:
def main():
    output_dir = create_output_directory("model_training_results")
    X_train, X_test, y_train, y_test = load_and_prepare_data()
    input_size = X_train.shape[1]  
    hidden_sizes = [256, 128, 64]  
    output_size = 3  
    batch_size = 64
    num_epochs = 50
    learning_rate = 0.001
    dropout_rate = 0.2
    
    print(f"Размер входного слоя: {input_size}")
    print(f"Архитектура сети: {input_size} -> {' -> '.join(map(str, hidden_sizes))} -> {output_size}")
    
    train_dataset = ChessMoveDataset(X_train, y_train)
    test_dataset = ChessMoveDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    model = ChessMoveNetwork(input_size, hidden_sizes, output_size, dropout_rate)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    start_time = time.time()
    train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, train_loader, test_loader, criterion, optimizer, 
        num_epochs, device, output_dir
    )
    training_time = time.time() - start_time
    visualize_training_results(
        train_losses, val_losses, train_accuracies, val_accuracies, output_dir
    )
    test_accuracy = evaluate_model(model, test_loader, device, output_dir)
    with open(output_dir / 'model_info.txt', 'w') as f:
        f.write("Информация о модели\n")
        f.write("==================\n\n")
        f.write(f"Архитектура:\n")
        f.write(f"- Входной слой: {input_size} нейронов\n")
        f.write(f"- Скрытые слои: {hidden_sizes}\n")
        f.write(f"- Выходной слой: {output_size} нейронов\n")
        f.write(f"- Dropout rate: {dropout_rate}\n\n")
        f.write(f"Параметры обучения:\n")
        f.write(f"- Batch size: {batch_size}\n")
        f.write(f"- Количество эпох: {num_epochs}\n")
        f.write(f"- Learning rate: {learning_rate}\n")
        f.write(f"- Оптимизатор: Adam\n")
        f.write(f"- Функция потерь: CrossEntropyLoss\n\n")
        f.write(f"Результаты:\n")
        f.write(f"- Время обучения: {training_time:.2f} секунд\n")
        f.write(f"- Точность на тестовой выборке: {test_accuracy:.4f}\n")
    
    print(f"\nОбучение завершено за {training_time:.2f} секунд")
    print(f"Результаты сохранены в {output_dir}")

In [33]:
if __name__ == "__main__":
    main() 

Создана директория для графиков: /Users/skolkoff/Desktop/StoneS_ЭП/model_training_results
Загружено 1358 строк данных
Размерность признаков: (1358, 26)
Размерность целевой переменной: (1358,)
Размер обучающей выборки: (1086, 26)
Размер тестовой выборки: (272, 26)
Размер входного слоя: 26
Архитектура сети: 26 -> 256 -> 128 -> 64 -> 3
Используется устройство: cpu

Начало обучения модели...
Epoch [1/50], Train Loss: 1.1155, Train Accuracy: 41.44%, Val Loss: 1.0068, Val Accuracy: 50.37%
Epoch [2/50], Train Loss: 0.9861, Train Accuracy: 50.00%, Val Loss: 0.9350, Val Accuracy: 54.78%
Epoch [3/50], Train Loss: 0.9341, Train Accuracy: 56.35%, Val Loss: 0.8756, Val Accuracy: 59.93%
Epoch [4/50], Train Loss: 0.8993, Train Accuracy: 58.93%, Val Loss: 0.8388, Val Accuracy: 62.87%
Epoch [5/50], Train Loss: 0.8543, Train Accuracy: 58.56%, Val Loss: 0.8017, Val Accuracy: 62.87%
Epoch [6/50], Train Loss: 0.8258, Train Accuracy: 60.68%, Val Loss: 0.8015, Val Accuracy: 64.34%
Epoch [7/50], Train Loss: 0

# neural network optimization

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [35]:
class ChessMoveDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels.values if hasattr(labels, 'values') else labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [36]:
class ChessMoveNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.2):
        super(ChessMoveNetwork, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_size),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [37]:
def create_output_directory(base_path="neural_network_figures"):
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для графиков: {output_dir.absolute()}")
    return output_dir

In [38]:
def load_data():
    """
    Загрузка и подготовка данных для обучения
    """
    df = pd.read_csv('preprocessed_data/processed_chess_data.csv')
    features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
               'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
               'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
               'square_control', 'game_phase', 'tempo', 'white_mobility', 'black_mobility',
               'white_king_safety', 'black_king_safety', 'white_development', 'black_development',
               'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square']
    
    available_features = [f for f in features if f in df.columns]
    if len(available_features) != len(features):
        print(f"Внимание: Доступно только {len(available_features)} из {len(features)} признаков")
        features = available_features
    
    X = df[features]
    y = df['result'].map({-1: 0, 0: 1, 1: 2})
    
    print(f"Размерность признаков: {X.shape}")
    print(f"Количество признаков: {len(features)}")
    
    return train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
def optimize_network_architecture(X_train, y_train, device):
    print("\n=== Оптимизация архитектуры нейронной сети ===")
    input_size = X_train.shape[1]
    print(f"Размер входного слоя: {input_size}")
    
    architectures = [
        {'hidden_sizes': [128, 64], 'dropout_rate': 0.2},
        {'hidden_sizes': [256, 128, 64], 'dropout_rate': 0.2},
        {'hidden_sizes': [512, 256, 128, 64], 'dropout_rate': 0.3},
        {'hidden_sizes': [256, 256, 128, 128, 64], 'dropout_rate': 0.25}
    ]
    
    results = []
    
    for arch in architectures:
        print(f"\nТестирование архитектуры: {arch['hidden_sizes']}")
        start_time = time.time()
        
        model = ChessMoveNetwork(
            input_size=input_size,
            hidden_sizes=arch['hidden_sizes'],
            output_size=3,
            dropout_rate=arch['dropout_rate']
        ).to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        train_dataset = ChessMoveDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        
        model.train()
        for epoch in range(10):  
            for features, labels in train_loader:
                features, labels = features.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            features = torch.FloatTensor(X_train).to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            y_train_tensor = torch.LongTensor(y_train.values if hasattr(y_train, 'values') else y_train)
            accuracy = (predicted.cpu() == y_train_tensor).sum().item() / len(y_train)
        
        training_time = time.time() - start_time
        
        results.append({
            'Архитектура': str(arch['hidden_sizes']),
            'Dropout': arch['dropout_rate'],
            'Точность': accuracy,
            'Время обучения (с)': training_time
        })
        
        print(f"Точность: {accuracy:.4f}")
        print(f"Время обучения: {training_time:.2f} секунд")
    
    return pd.DataFrame(results)

In [40]:
def visualize_optimization_results(results_df, output_dir):
    plt.figure(figsize=(12, 6))
    sns.barplot(data=results_df, x='Архитектура', y='Точность')
    plt.title('Сравнение точности различных архитектур')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_dir / 'architecture_accuracy_comparison.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    sns.barplot(data=results_df, x='Архитектура', y='Время обучения (с)')
    plt.title('Сравнение времени обучения различных архитектур')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_dir / 'architecture_training_time_comparison.png')
    plt.close()

In [41]:
if __name__ == "__main__":
    print("Начало процесса оптимизации нейронной сети...")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    X_train, X_test, y_train, y_test = load_data()
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    results_df = optimize_network_architecture(X_train, y_train, device)
    
    output_dir = create_output_directory()
    visualize_optimization_results(results_df, output_dir)
    
    results_df.to_csv('neural_network_optimization_results.csv', index=False)
    print("\nРезультаты сохранены в 'neural_network_optimization_results.csv'")
    print("Все графики сохранены.") 

Начало процесса оптимизации нейронной сети...
Используется устройство: cpu
Размерность признаков: (1358, 26)
Количество признаков: 26

=== Оптимизация архитектуры нейронной сети ===
Размер входного слоя: 26

Тестирование архитектуры: [128, 64]
Точность: 0.7339
Время обучения: 0.16 секунд

Тестирование архитектуры: [256, 128, 64]
Точность: 0.7983
Время обучения: 0.22 секунд

Тестирование архитектуры: [512, 256, 128, 64]
Точность: 0.7680
Время обучения: 0.35 секунд

Тестирование архитектуры: [256, 256, 128, 128, 64]
Точность: 0.7753
Время обучения: 0.36 секунд
Создана директория для графиков: /Users/skolkoff/Desktop/StoneS_ЭП/neural_network_figures

Результаты сохранены в 'neural_network_optimization_results.csv'
Все графики сохранены.


# detailed analysis

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time
from pathlib import Path

In [43]:
def create_output_directory(base_path="detailed_figures"):
    """
    Создание директории для сохранения графиков
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для графиков: {output_dir.absolute()}")
    return output_dir

In [44]:
def load_data():
    """
    Загрузка предобработанных данных
    """
    try:
        data_path = Path('preprocessed_data/processed_chess_data.csv')
        if data_path.exists():
            return pd.read_csv(data_path)
        
        data_path = Path('processed_chess_data.csv')
        if data_path.exists():
            return pd.read_csv(data_path)
            
        raise FileNotFoundError("Файл с данными не найден. Убедитесь, что файл processed_chess_data.csv существует в директории preprocessed_data/ или в корневой директории.")
    except Exception as e:
        print(f"Ошибка при загрузке данных: {str(e)}")
        raise

In [45]:
def perform_statistical_analysis(df, output_dir):
    """
    Выполнение статистического анализа
    """
    print("\n=== Статистический анализ ===")
    print("\nДоступные столбцы в данных:")
    print(df.columns.tolist())
    
    numeric_features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                       'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                       'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                       'square_control', 'game_phase', 'tempo', 'white_mobility', 'black_mobility',
                       'white_king_safety', 'black_king_safety', 'white_development', 'black_development',
                       'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square']
    
    available_features = [f for f in numeric_features if f in df.columns]
    if len(available_features) != len(numeric_features):
        print(f"Внимание: Доступно только {len(available_features)} из {len(numeric_features)} признаков")
        print(f"Недостающие признаки: {set(numeric_features) - set(available_features)}")
        numeric_features = available_features
    
    correlation_matrix = df[numeric_features].corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Корреляционная матрица числовых признаков')
    plt.tight_layout()
    plt.savefig(output_dir / 'correlation_matrix_detailed.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    for feature in numeric_features:
        plt.figure(figsize=(10, 6))
        sns.histplot(data=df, x=feature, bins=50)
        plt.title(f'Распределение {feature}')
        plt.savefig(output_dir / f'distribution_{feature}.png', dpi=300, bbox_inches='tight')
        plt.close()
    if 'mobility_diff' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='mobility_diff')
        plt.title('Распределение разности мобильности по результатам игр')
        plt.savefig(output_dir / 'mobility_by_result.png', dpi=300, bbox_inches='tight')
        plt.close()
    if 'center_control' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='center_control')
        plt.title('Распределение контроля центра по результатам игр')
        plt.savefig(output_dir / 'center_control_by_result.png', dpi=300, bbox_inches='tight')
        plt.close()
    if 'material_value' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='material_value')
        plt.title('Распределение материального преимущества по результатам игр')
        plt.savefig(output_dir / 'material_value_by_result.png', dpi=300, bbox_inches='tight')
        plt.close()
    if 'king_safety_diff' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df, x='result', y='king_safety_diff')
        plt.title('Распределение безопасности короля по результатам игр')
        plt.savefig(output_dir / 'king_safety_by_result.png', dpi=300, bbox_inches='tight')
        plt.close()

In [46]:
def perform_pca_analysis(df, output_dir):
    """
    Выполнение анализа главных компонент
    """
    print("\n=== Анализ главных компонент ===")
    
    features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                'square_control', 'tempo', 'white_mobility', 'black_mobility',
                'white_king_safety', 'black_king_safety', 'white_development', 'black_development',
                'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square']
    
    available_features = []
    for f in features:
        if f in df.columns and df[f].dtype in ['int64', 'float64']:
            available_features.append(f)
    
    if len(available_features) != len(features):
        print(f"Внимание: Доступно только {len(available_features)} числовых признаков из {len(features)}")
        features = available_features
    
    X = df[features]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
             np.cumsum(pca.explained_variance_ratio_), 'bo-')
    plt.xlabel('Количество компонент')
    plt.ylabel('Накопленная объясненная дисперсия')
    plt.title('Накопленная объясненная дисперсия по компонентам')
    plt.grid(True)
    plt.savefig(output_dir / 'pca_explained_variance.png', dpi=300, bbox_inches='tight')
    plt.close()
    plt.figure(figsize=(10, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
    plt.xlabel('Первая главная компонента')
    plt.ylabel('Вторая главная компонента')
    plt.title('Проекция данных на первые две главные компоненты')
    plt.savefig(output_dir / 'pca_visualization.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    feature_importance = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(len(features))],
        index=features
    )
    
    plt.figure(figsize=(12, 6))
    sns.heatmap(feature_importance, annot=True, cmap='coolwarm', center=0)
    plt.title('Вклад признаков в главные компоненты')
    plt.tight_layout()
    plt.savefig(output_dir / 'pca_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return feature_importance

In [47]:
def generate_detailed_report(df, feature_importance, output_dir):
    """
    Генерация детального отчета
    """
    with open(output_dir / 'detailed_analysis_report.txt', 'w', encoding='utf-8') as f:
        f.write("Детальный анализ шахматных данных\n")
        f.write("==============================\n\n")
        
        f.write("1. Общая информация о датасете\n")
        f.write("---------------------------\n")
        f.write(f"Количество позиций: {len(df)}\n")
        f.write(f"Количество признаков: {len(df.columns)}\n")
        f.write(f"Список признаков: {', '.join(df.columns)}\n\n")
        
        f.write("2. Распределение результатов\n")
        f.write("-------------------------\n")
        f.write(df['result'].value_counts().to_string())
        f.write("\n\n")
        
        f.write("3. Статистические характеристики\n")
        f.write("-----------------------------\n")
        f.write(df.describe().to_string())
        f.write("\n\n")
        
        f.write("4. Вклад признаков в главные компоненты\n")
        f.write("-----------------------------------\n")
        f.write(feature_importance.to_string())
        f.write("\n\n")
        
        f.write("5. Корреляции между признаками\n")
        f.write("---------------------------\n")
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
        exclude_columns = ['result', 'game_id', 'move_number', 'white_elo', 'black_elo']
        numeric_features = [col for col in numeric_columns if col not in exclude_columns]
        
        if numeric_features:
            f.write(df[numeric_features].corr().to_string())
        else:
            f.write("Числовые признаки не найдены")

In [48]:
def main():
    print("Начало детального анализа...")
    
    output_dir = create_output_directory("detailed_analysis_results")
    
    try:
        print("Загрузка данных...")
        df = load_data()
        print(f"Загружено {len(df)} строк данных")
        print("\nВыполнение статистического анализа...")
        perform_statistical_analysis(df, output_dir)
        print("\nВыполнение PCA анализа...")
        feature_importance = perform_pca_analysis(df, output_dir)
        print("\nГенерация отчета...")
        generate_detailed_report(df, feature_importance, output_dir)
        
        print(f"\nАнализ завершен. Все результаты сохранены в директории: {output_dir.absolute()}")
        
    except Exception as e:
        print(f"\nОшибка при выполнении анализа: {str(e)}")
        raise


In [49]:
if __name__ == "__main__":
    main() 

Начало детального анализа...
Создана директория для графиков: /Users/skolkoff/Desktop/StoneS_ЭП/detailed_analysis_results
Загрузка данных...
Загружено 1358 строк данных

Выполнение статистического анализа...

=== Статистический анализ ===

Доступные столбцы в данных:
['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff', 'material_value', 'center_control', 'extended_center_control', 'mobility_diff', 'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff', 'square_control', 'game_phase', 'tempo', 'white_mobility', 'black_mobility', 'white_king_safety', 'black_king_safety', 'white_development', 'black_development', 'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square', 'result', 'game_id', 'move_number', 'white_elo', 'black_elo', 'game_phase.1']

Выполнение PCA анализа...

=== Анализ главных компонент ===

Генерация отчета...

Анализ завершен. Все результаты сохранены в директории: /Users/skolkoff/Desktop/Sto

# neural network

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pathlib import Path
import time

In [51]:
class ChessDataset(Dataset):
    """
    Класс для работы с данными шахматных партий
    """
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        labels_array = labels.values if hasattr(labels, 'values') else labels
        self.labels = torch.LongTensor(labels_array.astype(np.int64))
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [52]:
class ImprovedChessNeuralNetwork(nn.Module):
    """
    Улучшенная нейронная сеть для анализа шахматных партий
    Упрощенная архитектура для предотвращения переобучения
    """
    def __init__(self, input_size, output_size, dropout_rate=0.4):
        super(ImprovedChessNeuralNetwork, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_size, max(32, input_size + 8)),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(max(32, input_size + 8), 24),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.7),  
            
            nn.Linear(24, output_size)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Инициализация весов для лучшей сходимости"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        return self.network(x)

In [53]:
class EarlyStopping:
    """Early stopping для предотвращения переобучения"""
    def __init__(self, patience=7, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


In [54]:
def create_output_directory(base_path="neural_network_results"):
    """
    Создание директории для сохранения результатов
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория для результатов: {output_dir.absolute()}")
    return output_dir

In [55]:
def load_and_prepare_data():
    """
    Загрузка и подготовка данных для обучения
    """
    try:
        df = pd.read_csv('preprocessed_data/processed_chess_data.csv')
        print(f"Загружено {len(df)} строк данных")
        
        features = ['pawn_diff', 'knight_diff', 'bishop_diff', 'rook_diff', 'queen_diff',
                   'material_value', 'center_control', 'extended_center_control', 'mobility_diff',
                   'king_safety_diff', 'development_diff', 'pawn_structure_diff', 'piece_square_diff',
                   'square_control', 'game_phase', 'tempo', 'white_mobility', 'black_mobility',
                   'white_king_safety', 'black_king_safety', 'white_development', 'black_development',
                   'white_pawn_structure', 'black_pawn_structure', 'white_piece_square', 'black_piece_square']
        
        available_features = [f for f in features if f in df.columns]
        if len(available_features) != len(features):
            print(f"Внимание: Доступно только {len(available_features)} из {len(features)} признаков")
            print(f"Недостающие признаки: {set(features) - set(available_features)}")
        
        X = df[available_features]
        y = df['result'].map({-1: 0, 0: 1, 1: 2})
        print(f"Размерность признаков: {X.shape}")
        print(f"Размерность целевой переменной: {y.shape}")
        
        from sklearn.utils import resample
        
        df_class_0 = df[y == 0]  
        df_class_1 = df[y == 1]  
        df_class_2 = df[y == 2]  
        
        print(f"Распределение классов до балансировки:")
        print(f"Победы черных: {len(df_class_0)}")
        print(f"Ничьи: {len(df_class_1)}")
        print(f"Победы белых: {len(df_class_2)}")
        
        min_class_size = min(len(df_class_0), len(df_class_1), len(df_class_2))
        target_size = min(min_class_size, 5000)  
        
        df_class_0_balanced = resample(df_class_0, n_samples=target_size, random_state=42)
        df_class_1_balanced = resample(df_class_1, n_samples=target_size, random_state=42)
        df_class_2_balanced = resample(df_class_2, n_samples=target_size, random_state=42)
        df_balanced = pd.concat([df_class_0_balanced, df_class_1_balanced, df_class_2_balanced])
        
        X = df_balanced[available_features]
        y = df_balanced['result'].map({-1: 0, 0: 1, 1: 2})
        
        print(f"Размер после балансировки: {X.shape}")
        print(f"Распределение классов после балансировки:")
        print(y.value_counts().sort_index())
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        print(f"Размер обучающей выборки: {X_train.shape}")
        print(f"Размер тестовой выборки: {X_test.shape}")
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test
    except Exception as e:
        print(f"Ошибка при загрузке данных: {str(e)}")
        raise

In [56]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                num_epochs, device, output_dir):
    """
    Обучение модели с early stopping и адаптивным learning rate
    """
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    best_val_accuracy = 0
    
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        scheduler.step(val_loss)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), output_dir / 'best_model.pth')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping на эпохе {epoch+1}")
            break
    
    return train_losses, val_losses, train_accuracies, val_accuracies

In [57]:
def visualize_training_results(train_losses, val_losses, train_accuracies, 
                             val_accuracies, output_dir):
    """
    Визуализация результатов обучения
    """
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss', alpha=0.8)
    plt.plot(val_losses, label='Validation Loss', alpha=0.8)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy', alpha=0.8)
    plt.plot(val_accuracies, label='Validation Accuracy', alpha=0.8)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir / 'training_plots.png', dpi=300, bbox_inches='tight')
    plt.close()

In [58]:
def analyze_network_performance(model, test_loader, device, output_dir):
    """
    Подробный анализ производительности сети
    """
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for features, labels in test_loader:
            features = features.to(device)
            outputs = model(features)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    from sklearn.metrics import confusion_matrix, classification_report
    
    cm = confusion_matrix(all_labels, all_predictions)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Черные побеждают', 'Ничья', 'Белые побеждают'],
                yticklabels=['Черные побеждают', 'Ничья', 'Белые побеждают'])
    plt.title('Confusion Matrix')
    plt.ylabel('Фактический результат')
    plt.xlabel('Предсказанный результат')
    plt.savefig(output_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    report = classification_report(all_labels, all_predictions, 
                                 target_names=['Черные побеждают', 'Ничья', 'Белые побеждают'])
    
    print(f"\nFinal Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print("\nClassification Report:")
    print(report)
    
    probabilities = np.array(all_probabilities)
    max_probs = np.max(probabilities, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.hist(max_probs, bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel('Максимальная вероятность предсказания')
    plt.ylabel('Количество предсказаний')
    plt.title('Распределение уверенности модели в предсказаниях')
    plt.grid(True, alpha=0.3)
    plt.savefig(output_dir / 'confidence_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return accuracy

In [59]:
def main():
    output_dir = create_output_directory()
    
    X_train, X_test, y_train, y_test = load_and_prepare_data()
    input_size = X_train.shape[1]
    output_size = 3
    batch_size = 64  
    num_epochs = 50  
    learning_rate = 0.01  
    dropout_rate = 0.2  
    
    print(f"Размер входного слоя: {input_size}")
    print(f"Упрощенная архитектура: {input_size} -> {max(32, input_size + 8)} -> 24 -> {output_size}")
    
    train_dataset = ChessDataset(X_train, y_train)
    test_dataset = ChessDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = ImprovedChessNeuralNetwork(input_size, output_size, dropout_rate)
    model = model.to(device)
    from sklearn.utils.class_weight import compute_class_weight
    
    unique_classes = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
    class_weights = np.power(class_weights, 0.7)  
    class_weights_tensor = torch.FloatTensor(class_weights).to(device)
    
    print(f"Веса классов (смягченные): {class_weights}")
    
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, 
                                                    patience=5)
    
    print("\nНачало обучения улучшенной модели...")
    start_time = time.time()
    
    train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, train_loader, test_loader, criterion, optimizer, scheduler,
        num_epochs, device, output_dir
    )
    training_time = time.time() - start_time
    print(f"\nОбучение завершено за {training_time:.2f} секунд")

    model.load_state_dict(torch.load(output_dir / 'best_model.pth'))
    visualize_training_results(
        train_losses, val_losses, train_accuracies, val_accuracies, output_dir
    )
    test_accuracy = analyze_network_performance(model, test_loader, device, output_dir)
    
    with open(output_dir / 'improved_model_info.txt', 'w', encoding='utf-8') as f:
        f.write("Улучшенная нейронная сеть - Отчет\n")
        f.write("=================================\n\n")
        f.write(" ПРИМЕНЕННЫЕ УЛУЧШЕНИЯ:\n")
        f.write("1. Упрощенная архитектура для предотвращения переобучения\n")
        f.write("2. Early stopping с patience=10\n")
        f.write("3. Gradient clipping для стабильности\n")
        f.write("4. Адаптивный learning rate scheduler\n")
        f.write("5. Улучшенная балансировка данных\n")
        f.write("6. Смягченные веса классов\n")
        f.write("7. Инициализация весов Xavier\n\n")
        f.write(f" АРХИТЕКТУРА:\n")
        f.write(f"- Входной слой: {input_size} нейронов\n")
        f.write(f"- Скрытый слой 1: {max(32, input_size + 8)} нейронов + ReLU + Dropout({dropout_rate})\n")
        f.write(f"- Скрытый слой 2: 24 нейрона + ReLU + Dropout({dropout_rate * 0.7:.2f})\n")
        f.write(f"- Выходной слой: {output_size} нейронов\n\n")
        f.write(f"- ПАРАМЕТРЫ ОБУЧЕНИЯ:\n")
        f.write(f"- Batch size: {batch_size}\n")
        f.write(f"- Максимум эпох: {num_epochs}\n")
        f.write(f"- Learning rate: {learning_rate}\n")
        f.write(f"- Оптимизатор: Adam с weight_decay=1e-4\n")
        f.write(f"- Scheduler: ReduceLROnPlateau\n")
        f.write(f"- Функция потерь: CrossEntropyLoss со смягченными весами\n\n")
        f.write(f"- РЕЗУЛЬТАТЫ:\n")
        f.write(f"- Время обучения: {training_time:.2f} секунд\n")
        f.write(f"- Количество эпох: {len(train_losses)}\n")
        f.write(f"- Лучшая валидационная точность: {max(val_accuracies):.2f}%\n")
        f.write(f"- Итоговая тестовая точность: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)\n\n")
        f.write(f"- ЦЕЛЬ: Превысить 70% точности\n")
        f.write(f"- Результат: {'ДОСТИГНУТА' if test_accuracy > 0.60 else 'НЕ ДОСТИГНУТА'}\n")


In [60]:
if __name__ == "__main__":
    main() 

Создана директория для результатов: /Users/skolkoff/Desktop/StoneS_ЭП/neural_network_results
Загружено 1358 строк данных
Размерность признаков: (1358, 26)
Размерность целевой переменной: (1358,)
Распределение классов до балансировки:
Победы черных: 446
Ничьи: 456
Победы белых: 456
Размер после балансировки: (1338, 26)
Распределение классов после балансировки:
result
0    446
1    446
2    446
Name: count, dtype: int64
Размер обучающей выборки: (1070, 26)
Размер тестовой выборки: (268, 26)
Размер входного слоя: 26
Упрощенная архитектура: 26 -> 34 -> 24 -> 3
Using device: cpu
Веса классов (смягченные): [0.99934631 0.99934631 1.00131049]

Начало обучения улучшенной модели...
Epoch [1/50], Train Loss: 1.0924, Train Accuracy: 44.67%, Val Loss: 0.9688, Val Accuracy: 54.10%
Epoch [2/50], Train Loss: 0.9814, Train Accuracy: 51.40%, Val Loss: 0.9432, Val Accuracy: 52.61%
Epoch [3/50], Train Loss: 0.9223, Train Accuracy: 55.79%, Val Loss: 0.8883, Val Accuracy: 58.21%
Epoch [4/50], Train Loss: 0.